#  TCP Traffic
<i>Adapted for use with FABRIC from [TCP Traffic](https://www.cs.unc.edu/Research/geni/geniEdu/03-TcpTraffic.html) and from [TCP vs. UDP](https://www.cs.unc.edu/Research/geni/geniEdu/07-TCPvsUDP.html) </i>
        
<b> Prerequisites  </b>
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-5 at https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md#section-1-get-started.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://github.com/fabric-testbed/teaching-materials/blob/main/Getting%20Started.md)

This experiment will provide you with the experience of generating and analyzing TCP flows using iperf. This experiment will also demonstrate the differences in UDP and TCP by exploring how the two protocols share the link when running simultaneously. You will use iperf to create the flow from a node to another and see how congestion control reacts once UDP traffic is has started

This is the second step in this assignment, to go to the previous step go to slice creation notebook or click [Here](./CreateSlice.ipynb)

In the first section the module gives students experience generating and analyzing TCP flows. Students will use iperf to create a flow and view the sawtooth behavior. A second flow will then be introduced to show how TCP flows share a link. (Iperf)

In the second section the module demonstrates differences in UDP and TCP by exploring how the two protocols share the link when running simultaneously. Students will use iperf to create a TCP flow from one node to another and UDP to create a flow in the other direction to show how TCP's congestion control reacts to UDP traffic.

## 1. Setup the Experiment
### 1.1 Reserve Resources

#### Import the Fabric API

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

####  Retrieve Slice
Create the slice at the [Create Slice Notebook](./CreateSlice.ipynb) and import it here.


In [ ]:
slice_name = 'TCP_Traffic_intro'
try:
    slice = fablib.get_slice(slice_name)
    print(f"Slice: {slice.get_name()}, {slice.get_state()}")
    slice.list_nodes()
except Exception as e:
    print(f"Get Slices Fail: {e}")
    

In [ ]:
for node in slice.get_nodes():
    print(f"nd {node.get_ssh_command()}")

### 1.2 Setup the node network

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.0.0.0/24")

Network = 'Lan'

In [ ]:
# configure nodes
try:
    A = slice.get_node('ND_A')
    B = slice.get_node('ND_B')
    
    #A's lan
    A_iface = A.get_interface(network_name=Network)  
    A_iface.ip_addr_add(addr='10.0.0.1', subnet=subnet)
    #B's Lan
    B_iface = B.get_interface(network_name=Network)  
    B_iface.ip_addr_add(addr='10.0.0.2', subnet=subnet)
    
    for node in slice.get_nodes():        
        stdout, stderr = node.execute(f'sudo ip link set dev enp7s0 up')
        
except Exception as e:
    print(f"Fail: {e}")

### 1.3 Instalation of tools for the tutorial
This includes "net tools" and "openvswich"

In [ ]:
# configure nodes
try:
    for node in slice.get_nodes():        
        node.execute("sudo apt update")
        node.execute("sudo apt install net-tools")
        node.execute("sudo apt install iperf")
except Exception as e:
    print(f"Fail: {e}")

### 1.4 Set up to use TCP to monitor transmitted packets command
This section is needed so that you canreferance information of a graph in the second part of the experiment

In [ ]:
# configure nodes
try:
    #upload scripts to make a graphic
    for nd in slice.get_nodes():
        nd.upload_file("./scripts/ss-data-analysis.R","ss-data-analysis.R")
        nd.upload_file("./scripts/ss-output.sh","ss-output.sh")
        nd.execute("sudo apt-get -y install moreutils r-base-core r-cran-ggplot2 r-cran-littler;sudo sysctl -w net.ipv4.tcp_no_metrics_save=1;chmod +x ss-output.sh ")
except Exception as e:
    print(f"Fail: {e}")

## 2. Run Experiment
<br><img src="./figures/TCPTrafficTop.png"><br>
### 2.1 Iperf in one direction
iperf is a tool for measuring TCP and UDP bandwidth performance. In this section we will measure from client to server.
1. In the Server SSH terminal, start the iperf with he following command: ```iperf -s``` 
2. In the Client SSH terminal run the iperf command by typing ```iperf -c <nodeA_ip> -i 10 -t 180```
<br> The "-s" indicates that the node will run in server mode, likewise the "-c" is client mode, The "-i 10" tells iperf to print updates every 10 seconds, and "-t 180" to run for 180 before closing the connection
<br> Running iperf can be done in ether direction you would just need to use the commands on the opposite terminals with the correct ip for the second command, You can all ways find the ip in linux using ether "ifconfig" or "ip a", the ip for your node in the network should be under "enp7s0"


### 2.2 TCP vs UDP
In this Exercise we will see the link that the TCP and UDP traffic share when they are running simultaneously, we will use iperf to generate the traffic between the nodes and see how the congestion control will affect the output.
1. Start 2 terminals on node A and a two terminals on node B
2. first we will start the servers on the first terminal of node A, run the following command to start the UDP server and the TCP servers ```iperf -s -i 15 & iperf -s -u -i 10``` the TCP server will print information every 15 seconds and the UDP server will print information every 10 seconds
3. On the second A terminal prepare to run the script ```./ss-output.sh``` this will catch packets so that you are able to analyze the traffic changes in this experiment.
4. Start the script on terminal A and then immediately run the following command ```iperf -c <nodeA> -t 90 ``` On the first node B terminal this start the tcp trafic and will run for 90 seconds (1.5 minutes)
6. after 30 seconds run the following command ```iperf -c <NodeA> -t 30 -u -b 200m``` on the second B terminal this will start sending UDP traffic at the same time as TCP traffic, this will run for a minute
<br><img src="./figures/TUT.png"><br>
7. once the traffic stops for both nodes stop the command on the second A terminal by hitting ```Ctrl-C``` This command should take about 3-5 more minutes depending on the hardware and site you used


In [ ]:
# configure nodes
try:
    A.execute("Rscript ss-data-analysis.R  ")
    A.download_file("Traffic-ss.svg","Traffic-ss.svg")
except Exception as e:
    print(f"Fail: {e}")

See the output [here](./Traffic-ss.svg)

9. Now that you have seen the traffic re run the experiment steps 2 - 8 with step 6's command with an increased bandwidth ```iperf -c <client> -t 30 -u -b 500m```
10. run the following cell to compare the traffic.

In [ ]:
try:
    A.execute("ls")
    A.execute("Rscript ss-data-analysis.R  ")
    A.download_file("Traffic2.svg","Traffic-ss.svg")
except Exception as e:
    print(f"Fail: {e}")

See the output [here](./Traffic2.svg)

## Continue to The Assignment notebook

Once You have completed this notebook you should be able to continue to the TCP Traffic Assignment Notebook, you can ether open it on the explorer or click [Here](./Assignment.ipynb) to open the next notebook.